In [ ]:
# リスト 5.2.5
# 学習済みWord2Vecデータのロード

import gensim

model = gensim.models.Word2Vec.load("ja.bin")
print(model)

In [ ]:
# リスト 5.2.6
# 学習済みWord2Vecの挙動を調べる

# 「世間」の特徴量ベクトルを調べる
print("「世間」の特徴量ベクトル")
print(model.wv["世間"])

# 「世間」の類似語を調べる
print()
print("「世間」の類似語")
for item, value in model.wv.most_similar("世間"):
    print(item, value)

In [ ]:
# リスト 5.2.6
# 学習済み Word2Vec の挙動を調べる
#
# 目的:
#   - 学習済みベクトル空間における語「世間」の分布表現（埋め込みベクトル）を確認し、
#     余弦類似度に基づく近傍語（most_similar）を一覧します。
#
# 前提:
#   - `model` は gensim の Word2Vec もしくは KeyedVectors を既にロード済みであること。
#     * gensim 4 系では、語ベクトル操作は `model.wv` (KeyedVectors) 経由で行います。
#   - ベクトルは学習目的（CBOW/Skip-gram with Negative Sampling）によりスケール不定です。
#     そのため語間の「近さ」の評価にはノルムに依存しない余弦類似度が一般的に用いられます。
#
# 注意:
#   - 語彙外 (OOV) の場合は `model.wv['世間']` や `most_similar("世間")` で KeyError が発生します。
#     日本語では前処理（分かち書き・NFKC 正規化・表記ゆれ吸収）に強く依存するため、
#     実運用では「語が辞書に存在するか」を事前に確認することを推奨します。
#       例) `'世間' in model.wv.key_to_index`
#
# 補足（理論メモ）:
#   - `most_similar` は内部で正規化済みベクトルを用い、(実質) 余弦類似度に基づく上位近傍を返します。
#   - 近傍語は「意味が似ている語」に限らず、同一文脈で出現しやすい共起語・主題語が混在します。
#     これは分布仮説（“You shall know a word by the company it keeps”）に基づく自然な挙動です。

# 「世間」の特徴量ベクトルを調べる（非正規化ベクトルの生値を表示します）
print("「世間」の特徴量ベクトル")
print(model.wv["世間"])  # OOV の場合は KeyError。実運用では事前チェックを推奨。

# 「世間」の類似語を調べる（余弦類似度に基づく上位近傍を表示します）
print()
print("「世間」の類似語")
for item, value in model.wv.most_similar("世間"):
    # value は [-1, 1] 程度のスコア（余弦類似度相当）。1 に近いほど類似と解釈します。
    print(item, value)

# 参考（任意・コメントアウト例）:
# # OOV を安全に扱う簡易ガード
# token = '世間'
# if token in model.wv.key_to_index:
#     vec = model.wv[token]
#     print(f'vector_size={model.wv.vector_size}, norm={np.linalg.norm(vec):.4f}')
#     for w, s in model.wv.most_similar(token, topn=10):
#         print(w, f'{s:.3f}')
# else:
#     print(f"[OOV] '{token}' は語彙に存在しません。前処理・表記ゆれを確認してください。")

In [ ]:
# リスト 5.2.7
# 「日本」→「東京」に対応する「フランス」→「X」をベクトル類推で求める
#
# 理論メモ:
# - Word2Vec の類推は 3CosAdd（Mikolov, 2013）の基準で求めるのが一般的。
#   目的は、v(X) ≈ v(東京) - v(日本) + v(フランス) を満たす語 X を
#   余弦類似度 cos(v(X), 目標ベクトル) が最大となるように探索すること。
# - gensim の `most_similar(positive=[...], negative=[...])` は内部で
#   3CosAdd を実装しており、余弦類似度トップ N を返す。
# - 3CosMul（掛け算版）もあり、頻度バイアスが軽減されるケースがある。
# - 注意: OOV（語彙外）・表記ゆれ・分かち書きに強く依存するため、
#         語彙に含まれるかを必ず事前に確認すること。
#
# 使い方:
# - `model` は gensim の Word2Vec/KeyedVectors をロード済みである前提。
# - トップ候補を `x_hat` として表示する。

# 入力語（正例=足し算 / 負例=引き算）
positive = ["東京", "フランス"]
negative = ["日本"]

# 事前チェック: 語彙外(OOV)がないか
oov = [w for w in (positive + negative) if w not in model.wv.key_to_index]
if oov:
    print(f"[OOV] モデル語彙に存在しない語: {oov}")
else:
    # 1) 3CosAdd（標準）: most_similar
    results_add = model.wv.most_similar(positive=positive, negative=negative, topn=10)

    # 2) 明示ベクトル演算（確認用）: v = v(東京) - v(日本) + v(フランス)
    target_vec = model.wv["東京"] - model.wv["日本"] + model.wv["フランス"]
    results_vec = model.wv.similar_by_vector(target_vec, topn=10)

    # 3) 3CosMul（参考）: 頻度バイアスを抑えたい場合に試す
    #    ※ コーパスにより挙動が異なるため、Add/Vec と結果が違うことがある
    results_mul = model.wv.most_similar_cosmul(
        positive=positive, negative=negative, topn=10
    )

    # 結果表示（トップ候補が推定 X）
    print("=== 3CosAdd (most_similar) の上位候補 ===")
    for w, s in results_add:
        print(f"{w}\t{s:.4f}")
    x_hat = results_add[0][0]
    print(f"\n推定 X (Add): {x_hat}")

    print("\n=== 明示ベクトル演算 similar_by_vector の上位候補 ===")
    for w, s in results_vec:
        print(f"{w}\t{s:.4f}")
    x_hat_vec = results_vec[0][0]
    print(f"\n推定 X (Vec): {x_hat_vec}")

    print("\n=== 3CosMul (most_similar_cosmul) の上位候補 ===")
    for w, s in results_mul:
        print(f"{w}\t{s:.4f}")
    x_hat_mul = results_mul[0][0]
    print(f"\n推定 X (Mul): {x_hat_mul}")

    # 補足:
    # - 3つの手法で推定 X が一致しない場合は、学習コーパスのドメイン/規模、前処理、品詞の混在、
    #   語の多義性の影響が考えられる。タスクに合わせて正規化や語彙統一を見直すと良い。

In [ ]:
# リスト 5.2.8
# 「男」→「女」に対応する「Y」→「妻」をベクトル類推で求める
#
# 理論メモ:
# - Word2Vec のアナロジー（類推）は 3CosAdd（Mikolov, 2013）を用いるのが一般的。
#   ここでは v(Y) ≈ v(妻) + v(男) − v(女) を満たす語 Y を，
#   余弦類似度 cos(v(Y), v(妻)+v(男)−v(女)) が最大となるよう探索する。
# - gensim の `most_similar(positive=[...], negative=[...])` は 3CosAdd を内部実装している。
# - 品詞や表記ゆれ，分かち書き（Janome/MeCab など）で語彙が一致しないと OOV（語彙外）になるため，
#   事前に語彙確認を行う。
# - 社会的バイアスに依存する結果が出ることがある（注意: 学習コーパスに依存）。
#
# 前提:
# - 変数 `model` は gensim の Word2Vec/KeyedVectors をロード済み。
#   例: `from gensim.models import KeyedVectors; model = KeyedVectors.load("ja.kv")`

from typing import List, Tuple


def check_oov(words: List[str]) -> List[str]:
    """語彙外(OOV)の単語を列挙するユーティリティ。"""
    return [w for w in words if w not in model.wv.key_to_index]


def print_results(title: str, pairs: List[Tuple[str, float]], topn: int = 10) -> None:
    """(語, 類似度) のリストを整形出力。"""
    print(f"\n=== {title} 上位{topn} ===")
    for w, s in pairs[:topn]:
        print(f"{w}\t{s:.4f}")


# 入力語（正例=足し算 / 負例=引き算）
positive = ["妻", "男"]
negative = ["女"]

# 1) OOV チェック
need = positive + negative
oov = check_oov(need)
if oov:
    print(f"[OOV] モデル語彙に存在しない語が含まれています: {oov}")
else:
    # 2) 3CosAdd（標準）: most_similar
    results_add = model.wv.most_similar(positive=positive, negative=negative, topn=10)

    # 3) 明示ベクトル演算（検算用）: v = v(妻) + v(男) − v(女)
    target_vec = model.wv["妻"] + model.wv["男"] - model.wv["女"]
    results_vec = model.wv.similar_by_vector(target_vec, topn=10)

    # 4) 3CosMul（参考）: 頻度バイアスを抑えたい場合に試す
    try:
        results_mul = model.wv.most_similar_cosmul(
            positive=positive, negative=negative, topn=10
        )
    except AttributeError:
        results_mul = []

    # 5) 結果表示（推定 Y は各手法のトップ）
    print_results("3CosAdd (most_similar)", results_add)
    y_add = results_add[0][0]
    print(f"\n推定 Y (Add): {y_add}")

    print_results("explicit vector → similar_by_vector", results_vec)
    y_vec = results_vec[0][0]
    print(f"\n推定 Y (Vec): {y_vec}")

    if results_mul:
        print_results("3CosMul (most_similar_cosmul)", results_mul)
        y_mul = results_mul[0][0]
        print(f"\n推定 Y (Mul): {y_mul}")

    # 補足（実務上のヒント）:
    # - 「夫」「主人」など複数候補が出る場合がある。目的に応じて人手ルールで正規化する。
    # - ドメイン語彙の不足が原因なら，学習コーパス拡充・分かち書き統一・サブワード手法（FastText）を検討。